# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

Import the required functionality and activate the debug logger.

In [ ]:
from atmodeller import debug_logger
from atmodeller.core import InteriorAtmosphereSystem, Planet
from atmodeller.constraints import SystemConstraint, MassConstraint, BufferedFugacityConstraint, SystemConstraints
from atmodeller.thermodynamics import ChemicalComponent, GasSpecies, StandardGibbsFreeEnergyOfFormation
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2, BasaltLibourelN2
from atmodeller.utilities import earth_oceans_to_kg
import numpy as np
import csv

debug_logger()

In [ ]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2()))
species

In [ ]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet, gibbs_data=StandardGibbsFreeEnergyOfFormation())

In [ ]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw: float = 2.8

# The nitrogen mass is constant.
mass_N: float = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H: float = earth_oceans_to_kg(number_ocean_moles[realisation])
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: SystemConstraints = SystemConstraints([
        MassConstraint(species="H", value=mass_H),
        MassConstraint(species="C", value=mass_C),
        MassConstraint(species="N", value=mass_N),
        BufferedFugacityConstraint(log10_shift=fo2_shifts[realisation])
    ])
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    interior_atmosphere.solve(constraints)
    out_realisation: dict[str,float] = interior_atmosphere.pressures_dict

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = fo2_shifts[realisation]

    out.append(out_realisation)

    filename: str = "atmodeller_monte_carlo_tutorial3.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

The simulation data is output as:

In [ ]:
out